In [ ]:
import numpy as np
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (
     FigureCanvasTkAgg)
import tkinter as tk
from tkinter import messagebox

# File paths
files = ['/Users/divyanshukumar/Downloads/m&m.csv', '/Users/divyanshukumar/Downloads/itc.csv', '/Users/divyanshukumar/Downloads/l&t.csv', '/Users/divyanshukumar/Downloads/hdfc.csv', '/Users/divyanshukumar/Downloads/sunpha.csv', '/Users/divyanshukumar/Downloads/tcs.csv']
dfs = []

for file in files:
    temp = pd.read_csv(file)
    temp.columns = ['Date', file.replace('.csv', '')]
    dfs.append(temp)

stocks = reduce(lambda left, right: pd.merge(left, right, on='Date'), dfs)

# Calculate returns for the last 12 months
returns_12_months = (stocks.iloc[0, 1:] - stocks.iloc[12, 1:]) / stocks.iloc[12, 1:]

# Genetic Algorithm for Portfolio Optimization
np.random.seed(0)

# Define the parameters
num_assets = returns_12_months.shape[0]
num_generations = 100
population_size = 50
mutation_rate = 0.1

# Define the fitness function
def fitness(weights, returns):
    if np.any(weights < 0):  # Check if any weights are negative
        return -np.inf  # Return negative infinity for invalid portfolios
    portfolio_return = np.sum(returns * weights)
    return -portfolio_return  # We want to maximize return, so negate the score

# Initialize the population
population = np.random.rand(population_size, num_assets)
population /= np.sum(population, axis=1, keepdims=True)  # Normalize the initial population

# Main genetic algorithm loop
for generation in range(num_generations):
    # Calculate fitness for each individual
    fitness_scores = np.array([fitness(individual, returns_12_months) for individual in population])

    # Select the top-performing individuals
    num_parents = int(population_size * 0.2)
    parents = population[np.argsort(fitness_scores)[:num_parents]]

    # Create a new generation through crossover and mutation
    new_population = []
    for _ in range(population_size - num_parents):
        parent_indices = np.random.choice(num_parents, 2, replace=False)
        parent1, parent2 = parents[parent_indices[0]], parents[parent_indices[1]]

        crossover_point = np.random.randint(num_assets)
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

        if np.random.rand() < mutation_rate:
            mutation_index = np.random.randint(num_assets)
            child[mutation_index] += np.random.randn() * 0.01  # Small random change

        child /= np.sum(child)  # Normalize
        new_population.append(child)

    # Combine parents and the new generation
    population = np.vstack((parents, new_population))

# Find the best portfolio in the final generation
final_fitness_scores = np.array([fitness(individual, returns_12_months) for individual in population])
best_portfolio = population[np.argmax(final_fitness_scores)]

root = tk.Tk()
root.title("Portfolio Optimization Results")

# Create a frame to hold the graph and printed output
frame = tk.Frame(root)
frame.pack(padx=20, pady=20)

# Plotting the graph of portfolio weights and returns
fig, ax1 = plt.subplots(figsize=(8, 6))

# Plot portfolio weights as a bar chart
ax1.bar(range(num_assets), best_portfolio, align='center', alpha=0.7)
ax1.set_xlabel('Asset')
ax1.set_ylabel('Weight', color='tab:blue')
ax1.tick_params('y', colors='tab:blue')

# Create a second y-axis for returns
ax2 = ax1.twinx()
ax2.plot(range(num_assets), returns_12_months, color='tab:red', marker='o')
ax2.set_ylabel('Returns', color='tab:red')
ax2.tick_params('y', colors='tab:red')

plt.title('Best Portfolio Weights and Returns (12 months)')

# Display the printed output in a label
result_text = f"Best Portfolio Weights:\n{best_portfolio}\nExpected Return (12 months): {np.sum(returns_12_months * best_portfolio)}"
output_label = tk.Label(frame, text=result_text, justify="left", padx=10, pady=10)
output_label.pack()

# Embed the Matplotlib plot in the Tkinter window
canvas = FigureCanvasTkAgg(fig, master=frame)
canvas.draw()
canvas.get_tk_widget().pack()

root.mainloop()

# Call the function to display portfolio results